In [1]:
import numpy as np
import math
from scipy.linalg import solve_sylvester, block_diag
from collections import deque

In [2]:
# Main functions used for the implementation of the ISS method for both the single and two-mode states

# Binomial term
def binomial(k, i, eta):
    return (math.factorial(i) / (math.factorial(k) * math.factorial(i - k))) * (eta ** (i - k)) * ((1 - eta) ** k)

# General matrix of the Kraus operators
def kraus(n, k, eta):
    n, k = int(n), int(k)

    # Phase encoding
    diag_phase = [np.exp(1j * 0 * i) for i in range(0, n + 1)]
    u_phase = np.diag(diag_phase)

    # Loss encoding
    diagonal_elements = [np.sqrt(binomial(k, i, eta)) for i in range(k, n + 1)]
    diagonal_matrix = np.diag(diagonal_elements)
    zeros_matrix = np.zeros((n - k + 1, k))
    b = np.hstack((zeros_matrix, diagonal_matrix))

    return b @ u_phase

# Eigenmatrix of the phase derivative of the Kraus operators
def gamma_phase(n, k):
    n, k = int(n), int(k)

    # Auxiliar matrices for the gamma matrices
    zeros_matrix = np.zeros((n - k + 1, k))
    ident_elements = [1 for i in range(k, n + 1)]
    ident_matrix = np.diag(ident_elements)
    l_transf = np.hstack((zeros_matrix, ident_matrix))
    r_transf = np.transpose(l_transf)

    # Gamma matrix for the phase
    diag_lamb0_phase = [1j * i for i in range(0, n + 1)]
    array_lamb0_phase = np.diag(diag_lamb0_phase)
    gamma_0_phase = np.matrix(array_lamb0_phase)

    return l_transf @ gamma_0_phase @ r_transf

# Eigenmatrix of the loss derivative of the Kraus operators
def gamma_loss(n, k, eta):
    n, k = int(n), int(k)

    # Auxiliar matrices for the gamma matrices
    zeros_matrix = np.zeros((n - k + 1, k))
    ident_elements = [1 for i in range(k, n + 1)]
    ident_matrix = np.diag(ident_elements)
    l_transf = np.hstack((zeros_matrix, ident_matrix))
    r_transf = np.transpose(l_transf)

    # Gamma matrix for the loss
    diag_lamb0_loss = [(i * (1 - eta) - k) for i in range(0, n + 1)]
    array_lamb0_loss = np.diag(diag_lamb0_loss)
    gamma_0_loss = np.matrix(array_lamb0_loss)

    return (l_transf @ gamma_0_loss @ r_transf) / (2 * eta * (1 - eta))

In [3]:
# Normalized M matrix for the single-mode state
def matrix_sm(ket, n, eta, x_phase, x_loss):
    n = int(n)

    m_phase_sm = np.zeros((n + 1, n + 1), dtype=complex)
    m_loss_sm = np.zeros((n + 1, n + 1), dtype=complex)

    for k in range(0, n + 1):
        
        # Density matrix and derivatives
        rho0 = ket @ np.conjugate(np.transpose(ket))
        rho = np.zeros((n + 1, n + 1), dtype=complex)
        dp_rho = np.zeros((n + 1, n + 1), dtype=complex)
        dq_rho = np.zeros((n + 1, n + 1), dtype=complex)

        for i in range(0, n + 1):
            
            # Kraus and Gamma matrices for the single-mode state
            zeros_v = np.zeros((i, n + 1))
            zeros_h = np.zeros((n - i + 1, i))
            kraus_sm = np.vstack((kraus(n, i, eta), zeros_v))
            gamma_phase_sm = np.vstack((np.hstack((gamma_phase(n, i), zeros_h)), zeros_v))
            gamma_loss_sm = np.vstack((np.hstack((gamma_loss(n, i, eta), zeros_h)), zeros_v))

            rho += kraus_sm @ rho0 @ np.conjugate(np.transpose(kraus_sm))
            dp_rho += gamma_phase_sm @ kraus_sm @ rho0 @ np.conjugate(
                np.transpose(kraus_sm)) + kraus_sm @ rho0 @ np.conjugate(np.transpose(gamma_phase_sm @ kraus_sm))
            dq_rho += gamma_loss_sm @ kraus_sm @ rho0 @ np.conjugate(
                np.transpose(kraus_sm)) + kraus_sm @ rho0 @ np.conjugate(np.transpose(gamma_loss_sm @ kraus_sm))

        # Converting single-mode matrices to two-mode matrices
        zeros_v = np.zeros((k, n + 1))
        zeros_h = np.zeros((n - k + 1, k))
        kraus_sm = np.vstack((kraus(n, k, eta), zeros_v))
        h_kraus_sm = np.conjugate(np.transpose(kraus_sm))
        gamma_phase_sm = np.vstack((np.hstack((gamma_phase(n, k), zeros_h)), zeros_v))
        gamma_loss_sm = np.vstack((np.hstack((gamma_loss(n, k, eta), zeros_h)), zeros_v))

        # SLD for phase obtained from the Slvester equation solver
        solution_sld_phase = solve_sylvester(rho, rho, 2 * dp_rho)
        sld_phase = (solution_sld_phase + np.conjugate(np.transpose(solution_sld_phase))) / 2

        # Calculation of m_phase_k
        dp_kraus_sm = gamma_phase_sm @ kraus_sm
        h_dp_kraus_sm = np.conjugate(np.transpose(dp_kraus_sm))
        m_phase_sm += 2 * (
                h_dp_kraus_sm @ sld_phase @ kraus_sm + h_kraus_sm @ sld_phase @ dp_kraus_sm) - h_kraus_sm @ sld_phase @ sld_phase @ kraus_sm

        # SLD for loss obtained from the Slvester equation solver
        solution_sld_loss = solve_sylvester(rho, rho, 2 * dq_rho)
        sld_loss = (solution_sld_loss + np.conjugate(np.transpose(solution_sld_loss))) / 2

        # Calculation of m_loss_k
        dq_kraus_sm = gamma_loss_sm @ kraus_sm
        h_dq_kraus_sm = np.conjugate(np.transpose(dq_kraus_sm))
        m_loss_sm += 2 * (
                    h_dq_kraus_sm @ sld_loss @ kraus_sm + h_kraus_sm @ sld_loss @ dq_kraus_sm) - h_kraus_sm @ sld_loss @ sld_loss @ kraus_sm

    # Matrix M
    m_sm = x_phase * m_phase_sm + x_loss * m_loss_sm
    
    return m_sm, rho, sld_phase, sld_loss

# Optimization for the single-mode state
def optimization_nphotons_sm(n, eta):

    reg = 1e-6  # Regularization factor
    stop = 5  # Number of collected result to ensure the stop condition
    tol = 0.001  # Condition for the last results to stop

    # ---------------------------------------- Optimization for the phase estimation ---------------------------------------- #

    # It is the phase normalization term at the normalized QFI
    
    # Initial guess for the phase estimation
    n = int(n)
    ket0_phase = np.ones((n + 1, 1)) / np.sqrt(n + 1)
    previous_qfi_phase = deque(maxlen=stop)

    iteration = 0
    while True:

        # M matrix for the phase estimation
        m_phase,_,_,_ = matrix_sm(ket0_phase, n, eta, 1, 0)

        # Eigenvectors for the phase estimation 
        eigenvalues_phase, eigenvectors_phase = np.linalg.eig(m_phase)
        index_largest_eigenvalue_phase = np.argmax(eigenvalues_phase)
        eigenvector_largest_phase = eigenvectors_phase[:, index_largest_eigenvalue_phase]
        ket0_phase = eigenvector_largest_phase.reshape(-1, 1) / np.linalg.norm(eigenvector_largest_phase)

        # Calculation of phase QFI
        rho0_phase = ket0_phase @ np.conjugate(np.transpose(ket0_phase))
        qfi_phase = np.trace(rho0_phase @ m_phase)

        # Stop condition
        previous_qfi_phase.append(qfi_phase)
        if len(previous_qfi_phase) >= 5:
            if all((qfi_phase - result) / result < tol for result in previous_qfi_phase):
                break

        iteration += 1
    
    # ------------------------------------- Optimization for the simultaneous estimation ------------------------------------ #    

    # Initial guess for the simultaneous estimation
    ket0 = np.ones((n + 1, 1)) / np.sqrt(n + 1)
    previous_qfi = deque(maxlen=stop)
    
    iteration = 0
    while True:

        # Normalized M matrix
        norm_phase = 1 / qfi_phase
        norm_loss = (eta * (1 - eta)) / n
        m_norm,_,_,_ = matrix_sm(ket0, n, eta, norm_phase, norm_loss)
        m_norm += reg * np.eye(m_norm.shape[0])

        # Eigenvectors for simultaneous estimation
        eigenvalues, eigenvectors = np.linalg.eig(m_norm)
        index_largest_eigenvalue = np.argmax(eigenvalues)
        eigenvector_largest = eigenvectors[:, index_largest_eigenvalue]
        ket0 = eigenvector_largest.reshape(-1, 1) / np.linalg.norm(eigenvector_largest)

        # Calculation of the normalized QFI
        rho0 = ket0 @ np.conjugate(np.transpose(ket0))
        qfi_norm = 0.5 * np.trace(rho0 @ m_norm)  

        # Stop condition
        previous_qfi.append(qfi_norm)
        if len(previous_qfi) == stop:
            average_qfi = sum(previous_qfi) / len(previous_qfi)
            if abs(qfi_norm - average_qfi) < tol:
                break

        iteration += 1

    return qfi_norm, qfi_phase, ket0, ket0_phase

In [4]:
# Normalized M matrix and SLDs for the two-mode state
def matrix_tm(ket, n, eta, x_phase, x_loss, reg=1e-10):
    n = int(n)

    m_phase_tm = np.zeros((n + 1, n + 1), dtype=complex)
    m_loss_tm = np.zeros((n + 1, n + 1), dtype=complex)

    rho_blocks = []
    sld_phase_blocks = []
    sld_loss_blocks = []

    for k in range(0, n + 1):
        
        # Kraus and Gamma matrices for the two-mode state
        kraus_tm = kraus(n, k, eta)
        h_kraus_tm = np.conjugate(np.transpose(kraus_tm))
        gamma_phase_tm = gamma_phase(n, k)
        gamma_loss_tm = gamma_loss(n, k, eta)

        # Density matrix at the output
        rho0 = ket @ np.conjugate(np.transpose(ket))
        rho_k = kraus_tm @ rho0 @ np.transpose(np.conjugate(kraus_tm))

        # SLD for phase
        pk = np.trace(rho_k)
        pk = pk if abs(pk) > reg else reg
        sld_phase_k = 2 * (gamma_phase_tm @ rho_k + rho_k @ np.conjugate(gamma_phase_tm)) / pk

        # Calculation of m_phase_k
        dp_kraus_tm = gamma_phase_tm @ kraus_tm
        h_dp_kraus_tm = np.conjugate(np.transpose(dp_kraus_tm))
        m_phase_tm += 2 * (h_dp_kraus_tm @ sld_phase_k @ kraus_tm + h_kraus_tm @ sld_phase_k @ dp_kraus_tm) - h_kraus_tm @ sld_phase_k @ sld_phase_k @ kraus_tm

        # SLD for loss
        der_pk = np.trace(
            np.conjugate(np.transpose(kraus_tm @ ket)) @ (np.conjugate(gamma_loss_tm) + gamma_loss_tm) @ kraus_tm @ ket)
        der_pk = der_pk if abs(der_pk) > reg else reg
        sld_loss_k = (2 * gamma_loss_tm @ rho_k + 2 * rho_k @ np.conjugate(gamma_loss_tm) - (der_pk / pk) * rho_k) / pk

        # Calculation of m_loss_k
        dq_kraus_tm = gamma_loss_tm @ kraus_tm
        h_dq_kraus_tm = np.conjugate(np.transpose(dq_kraus_tm))
        m_loss_tm += 2 * (h_dq_kraus_tm @ sld_loss_k @ kraus_tm + h_kraus_tm @ sld_loss_k @ dq_kraus_tm) - h_kraus_tm @ sld_loss_k @ sld_loss_k @ kraus_tm

        # Block matrices
        rho_blocks.append(rho_k)
        sld_phase_blocks.append(sld_phase_k)
        sld_loss_blocks.append(sld_loss_k)
    
    # Matrix M
    m_tm = x_phase * m_phase_tm + x_loss * m_loss_tm

    # Block matrices rho, sld_phase, sld_loss
    rho = block_diag(*rho_blocks)
    sld_phase = block_diag(*sld_phase_blocks)
    sld_loss = block_diag(*sld_loss_blocks)
    
    return m_tm, rho, sld_phase, sld_loss

# Optimization for the two-mode state
def optimization_nphotons_tm(n, eta):

    reg = 1e-6  # Regularization factor
    stop = 5  # Number of collected result to ensure the stop condition
    tol = 0.001  # Condition for the last results to stop

    # ---------------------------------------- Optimization for the phase estimation ---------------------------------------- #

    # It is the phase normalization term at the normalized QFI
    
    # Initial guess for the phase estimation
    n = int(n)
    ket0_phase = np.ones((n + 1, 1)) / np.sqrt(n + 1)
    previous_qfi_phase = deque(maxlen=stop)

    # Iteration
    iteration = 0
    while True:

        # Normalized M matrix
        m_phase,_,_,_ = matrix_tm(ket0_phase, n, eta, 1, 0)
        m_phase = (m_phase + np.conjugate(np.transpose(m_phase))) / 2

        # Eigenvectors for simultaneous estimation
        eigenvalues_phase, eigenvectors_phase = np.linalg.eig(m_phase)
        index_largest_eigenvalue_phase = np.argmax(eigenvalues_phase)
        eigenvector_largest_phase = eigenvectors_phase[:, index_largest_eigenvalue_phase]
        ket0_phase = eigenvector_largest_phase.reshape(-1, 1) / np.linalg.norm(eigenvector_largest_phase)

        # Calculation of normalized QFI 
        rho0_phase = ket0_phase @ np.conjugate(np.transpose(ket0_phase))
        qfi_phase = np.trace(rho0_phase @ m_phase)

        # Stop condition
        previous_qfi_phase.append(qfi_phase)
        if len(previous_qfi_phase) >= stop:
            if all((qfi_phase - result) / result < tol for result in previous_qfi_phase):
                break

        iteration += 1

    # ------------------------------------- Optimization for the simultaneous estimation ------------------------------------ #    
    
    # Initial guess for the simultaneous estimation
    ket0 = np.ones((n + 1, 1)) / np.sqrt(n + 1)
    previous_qfi = deque(maxlen=stop)

    # Iteration
    iteration = 0
    while True:

        # Normalized M matrix
        norm_phase = 1 / qfi_phase
        norm_loss = (eta * (1 - eta)) / n
        m_norm,_,_,_ = matrix_tm(ket0, n, eta, norm_phase, norm_loss)
        m_norm += reg * np.eye(m_norm.shape[0])

        # Eigenvectors for simultaneous estimation
        eigenvalues, eigenvectors = np.linalg.eig(m_norm)
        index_largest_eigenvalue = np.argmax(eigenvalues)
        eigenvector_largest = eigenvectors[:, index_largest_eigenvalue]
        ket0 = eigenvector_largest.reshape(-1, 1) / np.linalg.norm(eigenvector_largest)

        # Calculation of normalized QFIs
        rho0 = ket0 @ np.conjugate(np.transpose(ket0))
        qfi_norm = 0.5 * np.trace(rho0 @ m_norm)

        # Stop condition
        previous_qfi.append(qfi_norm)
        if len(previous_qfi) >= stop:
            if all((qfi_norm - result) / result < tol for result in previous_qfi):
                break

        iteration += 1

    return qfi_norm, qfi_phase, ket0, ket0_phase

In [5]:
# Auxiliar function to calculate: 
# 1. the off-diagonal terms of the QFI matrix
# 2. the average of the commutator of the SLDs
def qfi_aux(rho, qfi_phase, sld_phase, sld_loss):

    # Off diagonal QFI elements
    anticomm_sld = sld_phase @ sld_loss + sld_loss @ sld_phase
    qfi_phi_eta = 0.5 * np.trace(rho @ anticomm_sld)

    # Incompatibility term
    comm_sld = sld_phase @ sld_loss - sld_loss @ sld_phase
    incomp_phi_eta = np.trace(rho @ comm_sld)

    return qfi_phi_eta, incomp_phi_eta

In [6]:
%%time

# Examples of optimization for both the single-mode and two-mode state

eta = 0.1  # Value of the loss
n_values = [5, 9, 16, 28, 52, 94]  # Values of the photon number (in log scale)

print('Single-mode state:')

for n in n_values:
  
    qfi_norm, qfi_phase, ket, ket_phase = optimization_nphotons_sm(n, eta)
    m_sm, rho, sld_phase, sld_loss = matrix_sm(ket, n, eta, 1, 1)
    qfi_phi_eta, incomp_phi_eta = qfi_aux(rho, qfi_phase, sld_phase, sld_loss)
    print('n =', n, ', qfi_norm =', np.round(qfi_norm, 6), ', qfi_phase =', np.round(qfi_phase, 6), ', qfi_phi_eta =', np.round(qfi_phi_eta, 6), ', incomp_phi_eta =', np.round(incomp_phi_eta, 6))

print('Two-mode state:')

for n in n_values:

    qfi_norm, qfi_phase, ket0, ket0_phase = optimization_nphotons_tm(n, eta)
    m_tm, rho, sld_phase, sld_loss = matrix_tm(ket0, n, eta, 1, 1)    
    qfi_phi_eta, incomp_phi_eta = qfi_aux(rho, qfi_phase, sld_phase, sld_loss)
    print('n =', n, 'qfi_norm =', np.round(qfi_norm, 6), ', qfi_phase =', np.round(qfi_phase, 6), ', qfi_phi_eta =', np.round(qfi_phi_eta, 6), ', incomp_phi_eta =', np.round(incomp_phi_eta, 6))


Single-mode state:
n = 5 , qfi_norm = (0.825899+0j) , qfi_phase = (0.925537+0j) , qfi_phi_eta = -0j , incomp_phi_eta = -9.671325j
n = 9 , qfi_norm = (0.840325+0j) , qfi_phase = (1.918037+0j) , qfi_phi_eta = 0j , incomp_phi_eta = -19.933553j
n = 16 , qfi_norm = (0.854983+0j) , qfi_phase = (3.832696+0j) , qfi_phi_eta = 0j , incomp_phi_eta = -39.599709j
n = 28 , qfi_norm = (0.881872+0j) , qfi_phase = (7.387647+0j) , qfi_phi_eta = -0j , incomp_phi_eta = -76.927818j
n = 52 , qfi_norm = (0.891234+0j) , qfi_phase = (15.017175+0j) , qfi_phi_eta = 0j , incomp_phi_eta = -155.385917j
n = 94 , qfi_norm = (0.898953+0j) , qfi_phase = (29.181902+0j) , qfi_phi_eta = 0j , incomp_phi_eta = -298.721763j
Two-mode state:
n = 5 qfi_norm = (0.904197+0j) , qfi_phase = (1.253259+0j) , qfi_phi_eta = 0j , incomp_phi_eta = -12.172891j
n = 9 qfi_norm = (0.909805+0j) , qfi_phase = (2.378901+0j) , qfi_phi_eta = 0j , incomp_phi_eta = -23.150955j
n = 16 qfi_norm = (0.916163+0j) , qfi_phase = (4.492846+0j) , qfi_phi_et